In [70]:
 
 ! pip install fancyimpute   

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\debap\AppData\Local\Continuum\anaconda3\python.exe' 'C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\debap\AppData\Local\Temp\tmpkycyq_qe'
       cwd: C:\Users\debap\AppData\Local\Temp\pip-install-bb8_tfxw\cvxopt_291ea25f5bfb4d79b838c4c18bf32643
  Complete output (19 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-37
  creating build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\coneprog.py -> build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\cvxprog.py -> build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\info.py -> build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\misc.py -> build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\modeling.py -> build\lib.win-amd64-cpython-37\cvxopt
  copying src\python\msk.py -> build\lib


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29899 sha256=b0ffb768aecde4f04dd3110f452535169a060dff31d90894b52806bd9890c1cf
  Stored in directory: c:\users\debap\appdata\local\pip\cache\wheels\e3\04\06\a1a7d89ef4e631ce6268ea2d8cde04f7290651c1ff1025ce68
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=0681c5d9bb6e1a9f6f3c606a7130237defbb8ad4e49d009ab78ab2f47cfa7283
  Stored in directory: c:\users\debap\appdata\local\pip\cache\wheels\72\21\a8\a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built fancyimpute knnimpute
Failed to bui

  running build_ext
  building 'base' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for cvxopt
ERROR: Could not build wheels for cvxopt which use PEP 517 and cannot be installed directly


In [72]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [73]:
def impute_dataframe(df):
    imp = IterativeImputer(max_iter=10, random_state=0)
    imp.fit(df)
    imputed_df = pd.DataFrame(imp.transform(df), columns=df.columns)
    return imputed_df

## Ingesting the first week survey

In [138]:
first_week=pd.read_csv('SIIP+-+CS+173_February+7,+2023_13.45.csv')

## Printing Columns 

print(first_week.columns)

## Removing First Row 


first_row = first_week.iloc[:1]

first_week = first_week.iloc[1:]

## Creating a mapping file 
column_names = pd.DataFrame({'ColumnNames': first_week.columns})

col_map = pd.concat([first_row.reset_index(drop=True), column_names], axis=0)

## Removing Second Row 

first_week=first_week.reset_index(drop=True)
first_week=first_week.iloc[1:]

print('After removing first two rows: {} Rows'.format(first_week.shape[0]))

## Removing Missing UINs and NetIDs


first_week = first_week.dropna(subset=['Q48'])

print(' After Removing missing UINs :  {} Rows'.format(first_week.shape[0]))

## Checking for Missing values in questionaire columns 

columns_to_check = ['Q52', 'Q48', 'Q1_1', 'Q1_2',
       'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8', 'Q1_9', 'Q1_10',
       'Q1_11', 'Q1_12', 'Q1_13', 'Q1_14', 'Q1_15', 'Q1_16', 'Q2_1', 'Q2_2',
       'Q2_3', 'Q2_4', 'Q2_5', 'Q2_6', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
       'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18',
       'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28',
       'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38',
       'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46']

missing_threshold = len(columns_to_check) / 2
rows_with_missing_values = first_week[first_week[columns_to_check].isnull().sum(axis=1) > missing_threshold]

first_week=first_week[~(first_week[columns_to_check].isnull().sum(axis=1) > missing_threshold)]

print(' After removing rows with all the responses missing : {} Rows'.format(first_week.shape[0]))

## Removing Irrelevant Columns 

# Define the list of column names to drop
columns_to_drop = ['StartDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage']

# Drop the columns from the dataframe
first_week = first_week.drop(columns=columns_to_drop)

#print(first_week.shape)
## Replacing rows with missing values 

df_train=first_week.loc[:, ~first_week.columns.isin(['EndDate','Q48'])]
#print(df_train.shape)
end_date=pd.DataFrame(first_week[['EndDate','Q48']]).reset_index(drop=True)
#print(end_date.shape)
first_week=impute_dataframe(df_train)
#print(first_week.shape)
first_week=pd.concat([first_week,end_date],axis=1)
#print(first_week.shape)
#print(first_week.columns)

print(' After updating the missing values and removing irrelevant columns : {} Rows and columns'.format(first_week.shape))

### Removing @illinois.edu 

first_week['Q48'] = first_week['Q48'].str.replace('@illinois.edu', '')

first_week = first_week.reset_index(drop=True)

### Creating an isUIN Flag 

first_week['isUIN'] = first_week['Q48'].str.isnumeric()

### Removing those without consent 
first_week['fw_consent'] = (first_week['Q52'] == 1).astype(int)

### Updating Row Number and checking for duplicates

first_week['EndDate'] = pd.to_datetime(first_week['EndDate'])

first_week['rn'] = first_week.sort_values('EndDate').groupby('Q48').cumcount() + 1

first_week = first_week.sort_values(['Q48', 'EndDate'])
first_week['date_diff'] = first_week.groupby('Q48')['EndDate'].diff().dt.total_seconds() / 3600

value_counts = first_week['Q48'].value_counts()
first_week['isduplicate'] = first_week['Q48'].isin(value_counts[value_counts > 1].index)

first_week = first_week[~((first_week['isduplicate'] == True) & (first_week['rn'] > 1) & (first_week['date_diff'] < 24))]

first_week = first_week[((first_week['Q48'].duplicated(keep=False)) & (first_week['rn'] == 2)) | (~first_week['Q48'].duplicated(keep=False))]

#first_week[['Q48','EndDate','fw_consent','date_diff','rn','isduplicate']].sort_values(['Q48', 'EndDate'])

print(' After removing duplicates  : {} Rows '.format(first_week.shape[0]))

## Updating the Q48 Column and creating a special flag 

first_week['Q48'] = first_week['Q48'].str.lower()
first_week['IsName'] = ~first_week['Q48'].str.isalnum()

first_week.to_csv('first_week.csv',index=False)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q52', 'Q48', 'Q1_1', 'Q1_2',
       'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8', 'Q1_9', 'Q1_10',
       'Q1_11', 'Q1_12', 'Q1_13', 'Q1_14', 'Q1_15', 'Q1_16', 'Q2_1', 'Q2_2',
       'Q2_3', 'Q2_4', 'Q2_5', 'Q2_6', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8',
       'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18',
       'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27', 'Q28',
       'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38',
       'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46'],
      dtype='object')
After removing first two rows: 788 Rows
 After Removing missing UINs :  778 Rows
 After removing rows with 

C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:80: FutureWarning: The default value of regex will change from True to False in a future version.


## Ingesting the Post Exam Surveys

In [237]:
def combine_columns(df, column_prefixes):
    columns = [col for col in df.columns if any(col.startswith(prefix + ':') for prefix in column_prefixes)]
    combined_column = df[columns].idxmax(axis=1).str.split(':').str[1].str.strip()
    df[column_prefixes[0]] = combined_column
    df = df.drop(columns=columns)
    return df.reset_index(drop=True)


# def combine_columns(df, column_prefix):
#     columns = [col for col in df.columns if col.startswith(column_prefix)]
#     combined_column = df[columns].idxmax(axis=1).str.split(':').str[0].str.strip()
#     df[column_prefix] = combined_column
#     df = df.drop(columns=columns)
#     return df.reset_index(drop=True)

In [260]:
poe1=pd.read_excel('cs173postexam1spring2023.xlsx')
poe2=pd.read_excel('cs173postexam2spring2023.xlsx')
#print(poe1.columns)
print(poe2.shape)

## Removing duplicates for POE1


poe1['Submitted On'] = pd.to_datetime(poe1['Submitted On'])

poe1['rn'] = poe1.sort_values('Submitted On').groupby('Logged in User ID').cumcount() + 1

poe1 = poe1.sort_values(['Logged in User ID', 'Submitted On'])
poe1['date_diff'] = poe1.groupby('Logged in User ID')['Submitted On'].diff().dt.total_seconds() / 3600

value_counts = poe1['Logged in User ID'].value_counts()
poe1['isduplicate'] = poe1['Logged in User ID'].isin(value_counts[value_counts > 1].index)

poe1 = poe1[~((first_week['isduplicate'] == True) & (poe1['rn'] > 1) & (poe1['date_diff'] < 24))]

poe1 = poe1[((poe1['Logged in User ID'].duplicated(keep=False)) & (poe1['rn'] == 2)) | (~poe1['Logged in User ID'].duplicated(keep=False))]

#first_week[['Q48','EndDate','fw_consent','date_diff','rn','isduplicate']].sort_values(['Q48', 'EndDate'])

print(' After removing duplicates  from POE1 : {} Rows '.format(poe1.shape[0]))



## Removing duplicates for POE2 

poe2['Submitted On'] = pd.to_datetime(poe2['Submitted On'])

poe2['rn'] = poe2.sort_values('Submitted On').groupby('Logged in User ID').cumcount() + 1

poe2 = poe2.sort_values(['Logged in User ID', 'Submitted On'])
poe2['date_diff'] = poe2.groupby('Logged in User ID')['Submitted On'].diff().dt.total_seconds() / 3600

value_counts = poe2['Logged in User ID'].value_counts()
poe2['isduplicate'] = poe2['Logged in User ID'].isin(value_counts[value_counts > 1].index)

poe2 = poe2[~((first_week['isduplicate'] == True) & (poe2['rn'] > 1) & (poe2['date_diff'] < 24))]

poe2 = poe2[((poe2['Logged in User ID'].duplicated(keep=False)) & (poe2['rn'] == 2)) | (~poe2['Logged in User ID'].duplicated(keep=False))]

#first_week[['Q48','EndDate','fw_consent','date_diff','rn','isduplicate']].sort_values(['Q48', 'EndDate'])

print(' After removing duplicates from POE2  : {} Rows '.format(poe2.shape[0]))


## Drop Columns 

columns_to_drop = ['Staff Illinois College/Unit Name', 'Staff Illinois College/Unit Name', 'date_diff']
poe1 = poe1.drop(columns=columns_to_drop)
poe2=poe2.drop(columns=columns_to_drop)
#print(poe1.columns)


## Update Columns for Poe1
base_colname = 'Q'
num_cols = 18
columns = [f'{base_colname}{i+1}' for i in range(num_cols)]
for col in columns:
    #print(col)
    poe1 = combine_columns(poe1, [col])
    
    
## Update Columns for Poe2
base_colname = 'Q'
num_cols = 20
columns = [f'{base_colname}{i+1}' for i in range(num_cols) if i != 18]
for col in columns:
    #print(col)
    poe2 = combine_columns(poe2, [col])
# print(poe1.columns)
poe2.head()

## Removing illinois.edu 

#first_week['Q48'] = first_week['Q48'].str.replace('@illinois.edu', '')
poe1['Logged in User ID']=poe1['Logged in User ID'].str.replace('@illinois.edu', '')
poe2['Logged in User ID']=poe2['Logged in User ID'].str.replace('@illinois.edu', '')

## Renaming Columns in dataframe 
poe1.columns = 'poe1'+poe1.columns
poe2.columns = 'poe2'+poe2.columns

## Writing this to CSV 
poe1.to_csv('poe1.csv',index=False)
poe2.to_csv('poe2.csv',index=False)

(875, 95)


C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


 After removing duplicates  from POE1 : 790 Rows 


C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:80: FutureWarning: The default value of regex will change from True to False in a future version.
C:\Users\debap\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: The default value of regex will change from True to False in a future version.


 After removing duplicates from POE2  : 781 Rows 


## Grades

In [334]:
final=pd.read_csv('CS_173_Spring2023_gradebook_FINAL.csv')
midterm=pd.read_csv('CS_173_Spring2023_gradebook_MIDTERM.csv')

print(final.columns)
print(midterm.columns)

final=final.drop('Honors Credit',axis=1)

first_week=pd.read_csv('first_week.csv')
poe1=pd.read_csv('poe1.csv')
poe2=pd.read_csv('poe2.csv')
demo=pd.read_excel('Data_UIN.xlsx')
merit=pd.read_csv('meritstudents.csv')

#first_week, poe1, poe2, midterm and final
a = first_week[['Q48']].rename(columns={'Q48': 'netID'}).drop_duplicates()
b = pd.DataFrame({'netID': poe1['poe1Logged in User ID'].unique()})
c = pd.DataFrame({'netID': poe2['poe2Logged in User ID'].unique()})
d = midterm[['netID']].drop_duplicates()
e = pd.DataFrame({'netID': final['Net ID'].unique()})

result = pd.concat([a, b, c, d, e]).drop_duplicates()

first_week.columns = 'fw'+first_week.columns
midterm.columns = 'mdt'+midterm.columns 
final.columns='final'+final.columns 

first_week['Isfw']=1
poe1['IsPoe1']=1
poe2['IsPoe2']=1
midterm['Ismidterm']=1
final['IsFinal']=1


merged_df = pd.merge(result, first_week, left_on='netID', right_on='fwQ48', how='left')
merged_df = pd.merge(merged_df, poe1, left_on='netID', right_on='poe1Logged in User ID', how='left')
merged_df = pd.merge(merged_df, poe2, left_on='netID', right_on='poe2Logged in User ID', how='left')
merged_df = pd.merge(merged_df, midterm, left_on='netID',right_on='mdtnetID', how='left')
merged_df = pd.merge(merged_df, final, left_on='netID',right_on='finalNet ID', how='left')

merged_df[['Isfw', 'IsPoe1', 'IsPoe2', 'Ismidterm', 'IsFinal']] = merged_df[['Isfw', 'IsPoe1', 'IsPoe2', 'Ismidterm', 'IsFinal']].fillna(0)

## Adding demo and ismerit

demo=pd.read_excel('Data_UIN.xlsx')
merit=pd.read_csv('meritstudents.csv')
merit.columns='mrt'+merit.columns
demo.columns='dem'+demo.columns


merged_df = pd.merge(merged_df, demo, left_on='finalUIN', right_on='demUIN', how='left')
merged_df = pd.merge(merged_df, merit, left_on='finalUIN', right_on='mrtUIN', how='left')
merged_df['IsMerit'] = np.where(merged_df['mrtUIN'].isnull(), 0, 1)


cols = ['Isfw', 'IsPoe1', 'IsPoe2', 'Ismidterm', 'IsFinal','IsMerit'] + [col for col in merged_df.columns if col not in ['Isfw', 'IsPoe1', 'IsPoe2', 'Ismidterm', 'IsFinal','IsMerit']]
merged_df = merged_df.reindex(columns=cols)



merged_df.to_csv('cs173sp23masterdata.csv',index=False)

Index(['Net ID', 'UIN', 'Admit Term', 'Gender', 'Name', 'Email Address',
       'Credit', 'Level', 'Year', 'Subject', 'Number', 'Section', 'CRN',
       'Degree Name', 'Major 1 Name', 'College', 'Program Code',
       'Program Name', 'FERPA', 'Honors Credit', 'raw grade', 'letter grade'],
      dtype='object')
Index(['UIN', 'Overall raw grade', 'Letter grade', 'netID', 'gender'], dtype='object')


In [332]:
demo=pd.read_excel('Data_UIN.xlsx')
merit=pd.read_csv('meritstudents.csv')
merit.columns='mrt'+merit.columns
demo.columns='dem'+demo.columns

#merged_df['finalUIN'] = merged_df['finalUIN'].fillna(0)

# Perform left join on merged_df and demo
print(merged_df.shape[0])
merged_df = pd.merge(merged_df, demo, left_on='finalUIN', right_on='demUIN', how='left')
print(merged_df.shape[0])

# Perform left join on result and merit
print(merged_df.shape[0])
merged_df = pd.merge(merged_df, merit, left_on='finalUIN', right_on='mrtUIN', how='left')
print(merged_df.shape[0])
# Create new column IsMerit using np.where
merged_df['IsMerit'] = np.where(merged_df['mrtUIN'].isnull(), 0, 1)

861
861
861
861


0    818
1     43
Name: IsMerit, dtype: int64

In [326]:
display(merged_df.columns.tolist())

['Isfw',
 'IsPoe1',
 'IsPoe2',
 'Ismidterm',
 'IsFinal',
 'netID',
 'fwQ52',
 'fwQ1_1',
 'fwQ1_2',
 'fwQ1_3',
 'fwQ1_4',
 'fwQ1_5',
 'fwQ1_6',
 'fwQ1_7',
 'fwQ1_8',
 'fwQ1_9',
 'fwQ1_10',
 'fwQ1_11',
 'fwQ1_12',
 'fwQ1_13',
 'fwQ1_14',
 'fwQ1_15',
 'fwQ1_16',
 'fwQ2_1',
 'fwQ2_2',
 'fwQ2_3',
 'fwQ2_4',
 'fwQ2_5',
 'fwQ2_6',
 'fwQ3',
 'fwQ4',
 'fwQ5',
 'fwQ6',
 'fwQ7',
 'fwQ8',
 'fwQ9',
 'fwQ10',
 'fwQ11',
 'fwQ12',
 'fwQ13',
 'fwQ14',
 'fwQ15',
 'fwQ16',
 'fwQ17',
 'fwQ18',
 'fwQ19',
 'fwQ20',
 'fwQ21',
 'fwQ22',
 'fwQ23',
 'fwQ24',
 'fwQ25',
 'fwQ26',
 'fwQ27',
 'fwQ28',
 'fwQ29',
 'fwQ30',
 'fwQ31',
 'fwQ32',
 'fwQ33',
 'fwQ34',
 'fwQ35',
 'fwQ36',
 'fwQ37',
 'fwQ38',
 'fwQ39',
 'fwQ40',
 'fwQ41',
 'fwQ42',
 'fwQ43',
 'fwQ44',
 'fwQ45',
 'fwQ46',
 'fwEndDate',
 'fwQ48',
 'fwisUIN',
 'fwfw_consent',
 'fwrn',
 'fwdate_diff',
 'fwisduplicate',
 'fwIsName',
 'poe1Logged in User ID',
 'poe1Submitted On',
 'poe1First Name',
 'poe1Last Name',
 'poe1Is Illinois Staff',
 'poe1Staff Illinois D

In [327]:
print(demo.columns)
print('#######################')
print(merit.columns)


Index(['EDW_PERS_ID', 'UIN', 'FirstName', 'MiddleName', 'LastName', 'Gender',
       'Race', 'Stu_Attr', 'UnderRepDomestic', 'PermAddr1', 'PermCity',
       'PermState', 'PermZip', 'PermCountry', 'Duplicated EDW_PERS_ID',
       'TechGPA'],
      dtype='object')
#######################
Index(['Unnamed: 0', 'UIN'], dtype='object')


In [305]:
first_week.columns

Index(['Q52', 'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8',
       'Q1_9', 'Q1_10', 'Q1_11', 'Q1_12', 'Q1_13', 'Q1_14', 'Q1_15', 'Q1_16',
       'Q2_1', 'Q2_2', 'Q2_3', 'Q2_4', 'Q2_5', 'Q2_6', 'Q3', 'Q4', 'Q5', 'Q6',
       'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'EndDate', 'Q48', 'isUIN', 'fw_consent', 'rn', 'date_diff',
       'isduplicate', 'IsName'],
      dtype='object')

In [307]:
result=pd.DataFrame(result)
result.columns=['netID']

,fwQ52,fwQ1_1,fwQ1_2,fwQ1_3,fwQ1_4,fwQ1_5,fwQ1_6,fwQ1_7,fwQ1_8,fwQ1_9,...,fwQ46,fwEndDate,fwQ48,fwisUIN,fwfw_consent,fwrn,fwdate_diff,fwisduplicate,fwIsName,Isfw
0,4,3,3,3,3,3,3,3,3,3,...,1.0,28-01-2023 22:42,xl137,False,0,1,NaN,False,True,1
1,1,4,5,4,2,4,3,5,1,5,...,1.0,27-01-2023 02:31,ezrajc2,True,1,1,NaN,False,False,1
2,1,3,3,3,3,5,5,4,4,4,...,1.0,30-01-2023 21:09,jlope262,True,1,1,NaN,False,False,1
3,1,1,5,5,1,5,5,1,5,1,...,1.0,27-01-2023 15:03,hl87,True,1,1,NaN,False,False,1
4,1,2,3,3,2,3,4,4,2,3,...,1.0,28-01-2023 14:47,hannac4,True,1,1,NaN,False,False,1


Index(['netID', 'poe1Logged in User ID', 'poe1Submitted On', 'poe1First Name',
       'poe1Last Name', 'poe1Is Illinois Staff',
       'poe1Staff Illinois Department Name', 'poe1Is Illinois Student',
       'poe1Student Illinois College/Unit Name',
       'poe1Student Illinois Department Name', 'poe1rn', 'poe1isduplicate',
       'poe1Q1', 'poe1Q2', 'poe1Q3', 'poe1Q4', 'poe1Q5', 'poe1Q6', 'poe1Q7',
       'poe1Q8', 'poe1Q9', 'poe1Q10', 'poe1Q11', 'poe1Q12', 'poe1Q13',
       'poe1Q14', 'poe1Q15', 'poe1Q16', 'poe1Q17', 'poe1Q18', 'IsPoe1',
       'poe2Logged in User ID', 'poe2Submitted On', 'poe2First Name',
       'poe2Last Name', 'poe2Is Illinois Staff',
       'poe2Staff Illinois Department Name', 'poe2Is Illinois Student',
       'poe2Student Illinois College/Unit Name',
       'poe2Student Illinois Department Name',
       'poe2Q19:What percentage of your time doing these activities was spent with other students?',
       'poe2rn', 'poe2isduplicate', 'poe2Q1', 'poe2Q2', 'poe2Q3', '

In [245]:
def get_difference(df1: pd.DataFrame, col1: str, df2: pd.DataFrame, col2: str) -> list:
    difference = set(df1[col1]) - set(df2[col2])
    return list(difference)

In [274]:
print(len(get_difference(final,'Net ID',midterm,'netID'))) 
print(len(get_difference(midterm,'netID',final,'Net ID'))) 


0
43


In [248]:
len(get_difference(poe1,'poe1Logged in User ID',poe2,'poe2Logged in User ID')) ## 33
len(get_difference(poe2,'poe2Logged in User ID',poe1,'poe1Logged in User ID')) ## 24 



24

In [249]:
first_week=pd.read_csv('first_week.csv')
first_week.columns

Index(['Q52', 'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8',
       'Q1_9', 'Q1_10', 'Q1_11', 'Q1_12', 'Q1_13', 'Q1_14', 'Q1_15', 'Q1_16',
       'Q2_1', 'Q2_2', 'Q2_3', 'Q2_4', 'Q2_5', 'Q2_6', 'Q3', 'Q4', 'Q5', 'Q6',
       'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36',
       'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46',
       'EndDate', 'Q48', 'isUIN', 'fw_consent', 'rn', 'date_diff',
       'isduplicate', 'IsName'],
      dtype='object')

In [251]:
len(get_difference(poe1,'poe1Logged in User ID',first_week,'Q48')) ## 102 
len(get_difference(first_week,'Q48',poe1,'poe1Logged in User ID')) ## 59 
#len(get_difference(poe2,'poe2Logged in User ID',poe1,'poe1Logged in User ID')) ## 24 

59

In [253]:
len(get_difference(poe2,'poe2Logged in User ID',first_week,'Q48')) ## 98  
len(get_difference(first_week,'Q48',poe2,'poe2Logged in User ID')) ## 64  

64

In [255]:
poe1.to_csv('poe1.csv')
poe2.to_csv('poe2.csv')

409                 > 3 hours
721    > 1 hour but < 3 hours
288                  < 1 hour
382               Did not use
392    > 1 hour but < 3 hours
                ...          
268    > 1 hour but < 3 hours
620    > 1 hour but < 3 hours
163    > 1 hour but < 3 hours
8                   > 3 hours
206                 > 3 hours
Length: 790, dtype: object

In [175]:
poe1.Q1.head()

0                 > 3 hours
1    > 1 hour but < 3 hours
2                  < 1 hour
3               Did not use
4    > 1 hour but < 3 hours
Name: Q1, dtype: object